# Figure 5

In [ ]:
using DifferentialEquations
using ModelingToolkit
using MitochondrialDynamics
using MitochondrialDynamics: GlcConst, degavg, ΔΨm, Ca_c, Ca_m, t, G3P
using MitochondrialDynamics: Pyr, NADH_c, NADH_m, ATP_c, ADP_c, degavg
using MitochondrialDynamics.Utils: second, μM, mV, mM, Hz, minute
using Plots
pyplot()
Plots.pyrcparams["font.size"] = 14
Plots.pyrcparams["font.sans-serif"] = "Arial"
Plots.pyrcparams["font.family"] = "sans-serif"

In [ ]:
tend = 2000.0
@named sys = make_model(; glcrhs=10mM)
sssol = solve(SteadyStateProblem(sys, []), DynamicSS(Rodas5(), tspan=tend))
caavg = sssol[Ca_c]

In [ ]:
function cac_wave(t)
    ka_ca = (caavg - 0.09μM) * 1.5
    ca_r = 0.09μM
    period = 2minute
    x = 5 * modf(t / period)[1]
    ca_r + ka_ca * (x * exp(1 - x))^4
end

@register_symbolic cac_wave(t)
@named sysosci = make_model(; cacrhs=cac_wave(t), glcrhs=10mM)
sol = solve(ODEProblem(sysosci, sssol.u, tend))

In [ ]:
ts = LinRange(1520, tend, 200)
tsm = ts ./ 60

ca_c = sol(ts, idxs=Ca_c)
ca_m = sol(ts, idxs=Ca_m)
g3p = sol(ts, idxs=G3P)
pyr = sol(ts, idxs=Pyr)
nadh_c = sol(ts, idxs=NADH_c)
nadh_m = sol(ts, idxs=NADH_m)
atp_c = sol(ts, idxs=ATP_c)
adp_c = sol(ts, idxs=ADP_c)
td = atp_c.u ./ adp_c.u
dpsi = sol(ts, idxs=ΔΨm)
avgDeg = sol(ts, idxs=degavg)

In [ ]:
pl1 = plot(tsm, ca_c.u .* 1000, title="A", titlelocation=:left, label="Ca(cyto)")
plot!(pl1, tsm, ca_m.u .* 1000, label="Ca(mito)", xlabel="", ylabel="Conc. (μM)",
    legend=:right, xlims=(tsm[begin], tsm[end]), ylims=(0.0, 1.0))

In [ ]:
pl2 = plot(tsm, g3p.u .* 1000, title="B", titlelocation=:left, label="G3P")
plot!(pl2, tsm, nadh_c.u .* 1000, label="NADH (cyto)", xlabel="", ylabel="Conc. (μM)", legend=:right, xlims=(tsm[begin], tsm[end]))

In [ ]:
pl3 = plot(tsm, pyr.u .* 1000, title="C", titlelocation=:left, label="Pyr")
plot!(pl3, tsm, nadh_m.u .* 1000, label="NADH (mito)", xlabel="", ylabel="Conc. (μM)", legend=:right, xlims=(tsm[begin], tsm[end]))

In [ ]:
pl4 = plot(tsm, td, title="D", titlelocation=:left, label="ATP:ADP", xlabel="", legend=:right, xlims=(tsm[begin], tsm[end]))

In [ ]:
pl5 = plot(tsm, dpsi .* 1000, title="E", titlelocation=:left, label="ΔΨm", xlabel="", legend=:right, xlims=(tsm[begin], tsm[end]))

In [ ]:
pl6 = plot(tsm, avgDeg, title="F", titlelocation=:left, label="<k>", xlabel="Time (minute)", legend=:right, xlims=(tsm[begin], tsm[end]))

In [ ]:
plot(pl1, pl2, pl3, pl4, pl5, pl6, layout=(6, 1), size=(1000, 1200))

In [ ]:
# Uncomment to generate the pdf file 
# savefig("Fig5.pdf")